In [2]:
import cv2
import numpy as np
import os
import glob
import pandas as pd
from tqdm import tqdm

# Define the ImageNet mean and standard deviation
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406])
IMAGENET_STD = np.array([0.229, 0.224, 0.225])

def manually_normalize(image):
    image = image / 255.0  # Scale pixel values to [0, 1]
    image = (image - IMAGENET_MEAN) / IMAGENET_STD
    return image

def extract_frames(video_path, frame_count=5):
    cap = cv2.VideoCapture(video_path)
    frames = []
    try:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_indices = np.linspace(0, total_frames-1, frame_count, dtype=int)

        for frame_idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = cap.read()
            if ret:
                frame_resized = cv2.resize(frame, (224, 224))
                frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
                frame_normalized = manually_normalize(frame_rgb)
                frames.append(frame_normalized)
    finally:
        cap.release()
    
    return frames

# Define your category-to-folder mapping
folders = {
    "G": "Genuine",
    "Ps": "Still Printed Paper",
    "Pq": "Quivering Printed Paper",
    "Vl": "Lenovo LCD Display",
    "Vm": "Mac LCD Display",
    "Mc": "Paper Mask Cropped",
    "Mf": "Paper Mask Full",
    "Mu": "Paper Mask Upper Cut",
    "Ml": "Paper Mask Lower Cut"
}

# Define a folder to store the processed frames
processed_frame_folder = 'processed_frames'
os.makedirs(processed_frame_folder, exist_ok=True)

# Initialize a list to store frame paths and labels
frame_info = []

# Process each category
for label, folder in tqdm(folders.items(), desc='Processing categories'):
    video_files = glob.glob(os.path.join(folder, '*.mp4'))
    for video_file in tqdm(video_files, desc=f'Processing {label} videos'):
        frames = extract_frames(video_file)
        for idx, frame in enumerate(frames):
            # Define file path for the frame
            frame_filename = os.path.join(processed_frame_folder, f'{label}_{os.path.basename(video_file)}_frame_{idx}.npz')
            # Save the frame
            np.savez_compressed(frame_filename, frame)
            # Add the frame path and label to the manifest
            frame_info.append([frame_filename, label])

# Convert frame info to a DataFrame and save as CSV
frame_manifest = pd.DataFrame(frame_info, columns=['frame_path', 'label'])
frame_manifest.to_csv('frame_manifest.csv', index=False)


Processing G videos:  12%|█▏        | 110/897 [05:32<34:53,  2.66s/it]


Processing G videos:  25%|██▍       | 220/897 [09:34<15:22,  1.36s/it]


Processing G videos:  37%|███▋      | 330/897 [11:59<09:58,  1.06s/it]


Processing G videos:  49%|████▉     | 440/897 [14:16<09:16,  1.22s/it]


Processing G videos:  61%|██████▏   | 550/897 [16:33<07:00,  1.21s/it]


...

Processing Mu videos:  44%|████▎     | 174/400 [04:37<08:41,  2.31s/it]


Processing Mu videos:  71%|███████   | 283/400 [08:15<02:02,  1.05s/it]


Processing Mu videos:  98%|█████████▊| 392/400 [11:06<00:20,  2.62s/it]


Processing categories:  89%|████████▉ | 8/9 [1:36:11<10:13, 613.63s/it]
Processing Ml videos: 0it [00:00, ?it/s]
Processing categories: 100%|██████████| 9/9 [1:36:11<00:00, 641.24s/it]


NameError: name 'pd' is not defined

In [5]:
# Convert frame info to a DataFrame and save as CSV
frame_manifest = pd.DataFrame(frame_info, columns=['frame_path', 'label'])
frame_manifest.to_csv('frame_manifest.csv', index=False)